In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
con = duckdb.connect('../data/sakila.duckdb')

In [3]:
con.sql("SELECT * FROM film LIMIT 5").df()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00


## a) Movies longer than 3 hours

In [4]:
films = con.sql("FROM film;").df()

In [5]:
film_length = """--sql
SELECT
    length,
    title
FROM
    film
WHERE
    length > 180;
    """

In [6]:
df = con.sql(film_length).df()
df.head()

,length,title
0,181,ANALYZE HOOSIERS
1,182,BAKED CLEOPATRA
2,183,CATCH AMISTAD
3,185,CHICAGO NORTH
4,184,CONSPIRACY SPIRIT


In [7]:
film_length = films[films['length'] >= 180][['title', 'length']]
film_length

,title,length
15,ALLEY EVOLUTION,180
23,ANALYZE HOOSIERS,181
49,BAKED CLEOPATRA,182
127,CATCH AMISTAD,183
140,CHICAGO NORTH,185
173,CONFIDENTIAL INTERVIEW,180
179,CONSPIRACY SPIRIT,184
181,CONTROL ANTHEM,185
197,CRYSTAL BREAKING,184
211,DARN FORRESTER,185


## b) Which movies have the word "love" in its title? Show the following columns
- title
- rating
- length
- description

In [8]:
con.sql("""SELECT
    title,
    rating,
    length,
    description
FROM
    film
WHERE
    title ILIKE '%love%';""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


In [9]:
films[['title', 'rating', 'length', 'description']]
films[films['title'].str.contains('love', case=False)][['title', 'rating', 'length', 'description']]

,title,rating,length,description
373,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
447,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
448,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
457,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
510,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
534,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
535,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
536,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
537,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
851,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


## c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

Här tar jag fram kortaste, längsta, medellängd och medianlängd för filmer. Jag delar upp det i tre små queries för att det ska vara lätt att läsa.

### Shortest

Här kollar jag vilket längd som är kortast

In [10]:
con.sql("""--sql
SELECT 
    title,
    length
FROM
    film
ORDER BY
    length
LIMIT 5;""").df()

,title,length
0,ALIEN CENTER,46
1,IRON MOON,46
2,KWAI HOMEWARD,46
3,LABYRINTH LEAGUE,46
4,RIDGEMONT SUBMARINE,46


Ser att 46 är kortast så jag tar fram alla filmer som har lengtt som 46

In [11]:
shortest_films = con.sql("""--sql
SELECT 
    title,
    length
FROM
    film
WHERE
    length <= 46
ORDER BY
    length;""").df()

shortest_films

,title,length
0,ALIEN CENTER,46
1,IRON MOON,46
2,KWAI HOMEWARD,46
3,LABYRINTH LEAGUE,46
4,RIDGEMONT SUBMARINE,46


### Average och median

Här tar jag fram average och median längder

In [12]:
con.sql("""--sql
SELECT
    AVG(length) AS average_length,
    MEDIAN(length) AS median_length
FROM
    film;
""").df()

,average_length,median_length
0,115.272,114.0


### Longest

Här kollar jag vilken längd som är den längsta på filmerna

In [13]:
con.sql("""--sql
SELECT 
    title,
    length
FROM
    film
ORDER BY
    length DESC
LIMIT 5;""").df()

,title,length
0,CONTROL ANTHEM,185
1,DARN FORRESTER,185
2,CHICAGO NORTH,185
3,GANGS PRIDE,185
4,HOME PITY,185


Jag ser att 185 är den längsta filmen, så jag sorterar ut det

In [14]:
longest_films = con.sql("""--sql
SELECT 
    title,
    length
FROM
    film
WHERE
    length >= 185
ORDER BY
    length DESC;""").df()

longest_films

,title,length
0,CHICAGO NORTH,185
1,CONTROL ANTHEM,185
2,DARN FORRESTER,185
3,GANGS PRIDE,185
4,HOME PITY,185
5,MUSCLE BRIGHT,185
6,POND SEATTLE,185
7,SOLDIERS EVOLUTION,185
8,SWEET BROTHERHOOD,185
9,WORST BANGER,185


## d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie. The Manager wants to know the 10 most expensive movies to rent per day.

Här räknar jag ut kostnaden per dag för varje film genom att dividera rental_rate med rental_duration. Sedan sorterar jag filmerna fallande och visar de 10 dyraste per dag

In [15]:
most_expensive_movies_per_day = con.sql("""--sql
SELECT
    title,
    rental_rate,
    rental_duration,
    rental_rate / rental_duration AS cost_per_day
FROM
    film
ORDER BY
    cost_per_day DESC
LIMIT
    10;
""").df()

most_expensive_movies_per_day

,title,rental_rate,rental_duration,cost_per_day
0,BACKLASH UNDEFEATED,4.99,3,1.663333
1,BILKO ANONYMOUS,4.99,3,1.663333
2,BEAST HUNCHBACK,4.99,3,1.663333
3,AUTUMN CROW,4.99,3,1.663333
4,ACE GOLDFINGER,4.99,3,1.663333
5,CARIBBEAN LIBERTY,4.99,3,1.663333
6,BEHAVIOR RUNAWAY,4.99,3,1.663333
7,CASPER DRAGONFLY,4.99,3,1.663333
8,AMERICAN CIRCUS,4.99,3,1.663333
9,CASUALTIES ENCINO,4.99,3,1.663333


## e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

In [48]:
con.sql("""--sql
SELECT
    (first_name || ' ' || last_name) AS name,
    -- fa.actor_id,
    COUNT(fa.film_id) AS num_films
FROM
    actor a
JOIN
    film_actor fa ON a.actor_id = fa.actor_id
GROUP BY
    fa.actor_id, name
ORDER BY
    num_films DESC
LIMIT 10;

""").df()

,name,num_films
0,GINA DEGENERES,42
1,WALTER TORN,41
2,MARY KEITEL,40
3,MATTHEW CARREY,39
4,SANDRA KILMER,37
5,SCARLETT DAMON,36
6,UMA WOOD,35
7,VIVIEN BASINGER,35
8,GROUCHO DUNST,35
9,VAL BOLGER,35
